<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
OPTIONS_MODEL_PATH =  os.path.join(NOMEROFF_NET_DIR, "models/numberplate_kz_options_2019_04_17.h5")

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.pb
mode = "gpu"
OCR_NP_UKR_TEXT = os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ua_16-{}.h5".format(mode))
OCR_NP_EU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_eu_2-{}.h5".format(mode))
OCR_NP_RU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ru_3-{}.h5".format(mode))
OCR_NP_KZ_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_kz_3-{}.h5".format(mode))

sys.path.append(NOMEROFF_NET_DIR)

Also you may use .pb models for options detectors or OCR detectors. 
More about conversion and preparing for inference withit https://github.com/ria-com/nomeroff-net/blob/master/examples/inference_convert.ipynb.

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector({
    "class_region": ["xx_unknown", "eu_ua_2015", "eu_ua_2004", "eu_ua_1995", "eu", "xx_transit", "ru", "kz"]
})
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    },
    "ru": {
        "for_regions": ["ru"],
        "model_path": OCR_NP_RU_TEXT
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": OCR_NP_KZ_TEXT
    }
})

Using TensorFlow backend.
W0425 18:39:49.072923 139837171857216 deprecation_wrapper.py:119] From /var/www/nomeroff-net/NomeroffNet/Detector.py:12: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0425 18:39:49.073378 139837171857216 deprecation_wrapper.py:119] From /var/www/nomeroff-net/NomeroffNet/Detector.py:14: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0425 18:39:49.845840 139837171857216 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0425 18:39:49.847873 139837171857216 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0425 18:39:49.849502 139837171857216 deprecation_wrapper.py:119] From /usr/local/lib/p

In [4]:
# Walking through the ./examples/images/ directory and checking each of the images for license plates.
rootDir = 'images.kz/'

max_img_w = 1600
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        baseName = os.path.splitext(os.path.basename(fname))[0]
        img_path = os.path.join(dirName, fname)
        print(img_path)
        img = mpimg.imread(img_path)
        #plt.axis("off")
        #plt.imshow(img)
        #plt.show()
         
        # corect size for better speed
        img_w = img.shape[1]
        img_h = img.shape[0]
        img_w_r = 1
        img_h_r = 1
        if img_w > max_img_w:
            resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
            img_w_r = img_w/max_img_w
            img_h_r = img_h/(max_img_w/img_w*img_h)
        else:
            resized_img = img

        NP = nnet.detect([resized_img]) 
        
        # Generate image mask.
        cv_img_masks = await filters.cv_img_mask_async(NP)
            
        # Detect points.
        arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
        #print(arrPoints)
        arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
        arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r
        
        # cut zones
        zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
        toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)
    
        # find standart
        regionIds, stateIds = optionsDetector.predict(zones)
        regionNames = optionsDetector.getRegionLabels(regionIds)
        foundNumber=0
        for zone, points, regionId in zip(toShowZones, arrPoints, regionIds):
            #plt.axis("off")
            if regionId != 0 and foundNumber != 1:
                mpimg.imsave("result/{}.png".format(baseName), zone)
                foundNumber=1
                #plt.imshow(zone)
            #plt.show()
               

        #print(regionNames)

        # find text with postprocessing by standart  
        textArr = textDetector.predict(zones, regionNames)
        textArr = await textPostprocessingAsync(textArr, regionNames)
        #print(textArr)

images.kz/782CPB02.jpg
images.kz/P732OZM.jpg
images.kz/676DIA09.jpg
images.kz/925KOA10.jpg
images.kz/746FOA07.jpg
images.kz/973IGA02.jpg
images.kz/777CAA04.jpg
images.kz/397UZA01.jpg
images.kz/P659WDM.jpg
images.kz/398CEB05.jpg
images.kz/204AAA04.jpg
images.kz/985LAA02.jpg
images.kz/277HHA16.jpg
images.kz/F264KPN.jpg
images.kz/070AMC04.jpg
images.kz/090OAA05.jpg
images.kz/P946YRM.jpg
images.kz/101AST01.jpg
images.kz/039RAZ17.jpg
images.kz/255ERA15.jpg
images.kz/490BAZ15.jpg
images.kz/T176NOM.jpg
images.kz/286CLA02.jpg
images.kz/082KZA10.jpg
images.kz/902ANZ05.jpg
images.kz/305AT02.jpg
images.kz/137RK11.jpg
images.kz/198DMA03.jpg
images.kz/083CEA11.jpg
images.kz/010OQQ01.jpg
images.kz/419ACZ17.jpg
images.kz/D284XMM.jpg
images.kz/P302SFM.jpg
images.kz/062MAZ04.jpg
images.kz/F155CYM.jpg
images.kz/361BVA04.jpg
images.kz/808OOO04.jpg
images.kz/032FCB05.jpg
images.kz/Z549DD.jpg
images.kz/903AHA10.jpg
images.kz/623BZA07.jpg
images.kz/948AE01.jpg
images.kz/771LOA10.jpg
images.kz/979AYA16.jpg
i

images.kz/860DS02.jpg
images.kz/274HDA10.jpg
images.kz/A319AZP.jpg
images.kz/188VBZ06.jpg
images.kz/212HLA16.jpg
images.kz/858DLA11.jpg
images.kz/985HTB13.jpg
images.kz/788CFB05.jpg
images.kz/A053BTO.jpg
images.kz/909DNA04.jpg
images.kz/777AD16.jpg
images.kz/345CHA14.jpg
images.kz/016DRB13.jpg
images.kz/P158XOM.jpg
images.kz/C740HFM.jpg
images.kz/108CV02.jpg
images.kz/730FTA11.jpg
images.kz/864CTA11.jpg
images.kz/223PCA01.jpg
images.kz/070LAU01.jpg
images.kz/304AAZ10.jpg
images.kz/230LCA13.jpg
images.kz/039PKA01.jpg
images.kz/164HTA06.jpg
images.kz/981LFA02.jpg
images.kz/907VDZ02.jpg
images.kz/415URA02.jpg
images.kz/710KKA16.jpg
images.kz/Z173SZM.jpg
images.kz/207CYA10.jpg
images.kz/504GGA02.jpg
images.kz/358FBB02.jpg
images.kz/311RAA06.jpg
images.kz/947DPA07.jpg
images.kz/128BBB13.jpg
images.kz/770LLA04.jpg
images.kz/499CIA09.jpg
images.kz/105HSA16.jpg
images.kz/P823NCM.jpg
images.kz/040CEA11.jpg
images.kz/727VRZ03.jpg
images.kz/896BUB02.jpg
images.kz/114AK13.jpg
images.kz/295KXZ05.jp

images.kz/T714UAM.jpg
images.kz/707AW04.jpg
images.kz/870PAA01.jpg
images.kz/734CRA11.jpg
images.kz/165AZA11.jpg
images.kz/F507AON.jpg
images.kz/979AE06.jpg
images.kz/334DAA11.jpg
images.kz/733FAA10.jpg
images.kz/303CPA15.jpg
images.kz/742EKA15.jpg
images.kz/762SRA01.jpg
images.kz/021DWA14.jpg
images.kz/500AAA07.jpg
images.kz/696UNA09.jpg
images.kz/462FLB02.jpg
images.kz/985LKA05.jpg
images.kz/T829TOM.jpg
images.kz/222HHA12.jpg
images.kz/266BXA11.jpg
images.kz/120DOA14.jpg
images.kz/T889SFM.jpg
images.kz/636VIP15.jpg
images.kz/934BNZ02.jpg
images.kz/130RAA10.jpg
images.kz/839FAA10.jpg
images.kz/P390ATM.jpg
images.kz/117GOB02.jpg
images.kz/871WEA02.jpg
images.kz/577HXA14.jpg
images.kz/680HLA07.jpg
images.kz/727AR04.jpg
images.kz/070RPA10.jpg
images.kz/098YEA11.jpg
images.kz/859VAZ04.jpg
images.kz/457FCB02.jpg
images.kz/352KNA10.jpg
images.kz/847BR02.jpg
images.kz/548AL10.jpg
images.kz/102AMZ01.jpg
images.kz/253LCA10.jpg
images.kz/651OAA01.jpg
images.kz/B728HRO.jpg
images.kz/084CAA11.jpg

images.kz/051EWA14.jpg
images.kz/869NHA16.jpg
images.kz/502VCZ10.jpg
images.kz/859EMA08.jpg
images.kz/144BLB05.jpg
images.kz/059DMA10.jpg
images.kz/888AAA06.jpg
images.kz/052PAZ09.jpg
images.kz/880VXA01.jpg
images.kz/T777AXM.jpg
images.kz/030CHA14.jpg
images.kz/282ATA15.jpg
images.kz/242KFA10.jpg
images.kz/921AE07.jpg
images.kz/T689USM.jpg
images.kz/317VCZ10.jpg
images.kz/864UHA05.jpg
images.kz/908NHA05.jpg
images.kz/201AHA17.jpg
images.kz/523ABZ10.jpg
images.kz/699FAA10.jpg
images.kz/461KLA10.jpg
images.kz/467ABZ10.jpg
images.kz/672YYA02.jpg
images.kz/A147XXO.jpg
images.kz/075FUB05.jpg
images.kz/817AHA07.jpg
images.kz/933VAZ08.jpg
images.kz/125CLA11.jpg
images.kz/097YNA13.jpg
images.kz/652MXB13.jpg
images.kz/947KSA10.jpg
images.kz/019BBB01.jpg
images.kz/043PBZ03.jpg
images.kz/332DFA15.jpg
images.kz/189EWA10.jpg
images.kz/762NRA16.jpg
images.kz/570ABB09.jpg
images.kz/A310CK.jpg
images.kz/001CS15.jpg
images.kz/353BOZ02.jpg
images.kz/684EHA01.jpg
images.kz/979DBA10.jpg
images.kz/940FVA03

images.kz/121KYA10.jpg
images.kz/070PKA05.jpg
images.kz/046BAA11.jpg
images.kz/448DMA15.jpg
images.kz/027AYA13.jpg
images.kz/P837VMM.jpg
images.kz/D001SP.jpg
images.kz/101RR10.jpg
images.kz/643CAA11.jpg
images.kz/A893UPO.jpg
images.kz/B172YKN.jpg
images.kz/111VUA02.jpg
images.kz/001ZUG02.jpg
images.kz/958AH12.jpg
images.kz/965ZKZ05.jpg
images.kz/801YDA11.jpg
images.kz/010BUS10.jpg
images.kz/022PAZ11.jpg
images.kz/118VXA05.jpg
images.kz/828DOA12.jpg
images.kz/443HMA10.jpg
images.kz/061DVA15.jpg
images.kz/138WEA05.jpg
images.kz/845VDZ05.jpg
images.kz/X099FHM.jpg
images.kz/658AQA02.jpg
images.kz/212BFA03.jpg
images.kz/881KXA04.jpg
images.kz/244LZA04.jpg
images.kz/551HMA01.jpg
images.kz/942DAA16.jpg
images.kz/341AP07.jpg
images.kz/162AXA05.jpg
images.kz/P775WVM.jpg
images.kz/500RRR10.jpg
images.kz/F873CBN.jpg
images.kz/F385KDN.jpg
images.kz/926CWA15.jpg
images.kz/111MMM10.jpg
images.kz/080PAZ04.jpg
images.kz/060AAA12.jpg
images.kz/143UBA13.jpg
images.kz/734HMA10.jpg
images.kz/220AKA12.jpg


...

images.kz/388AAZ10.jpg
images.kz/814BSZ02.jpg
images.kz/942BZA15.jpg
images.kz/836BXB02.jpg
images.kz/T958PMM.jpg
images.kz/795KZA04.jpg
images.kz/881AL10.jpg
images.kz/660AOA10.jpg
images.kz/P301YEM.jpg
images.kz/505LCA01.jpg
images.kz/660HKA02.jpg
images.kz/650PKA13.jpg
images.kz/B033KPT.jpg
images.kz/656CCA06.jpg
images.kz/032AH16.jpg
images.kz/304AXA11.jpg
images.kz/088BRO04.jpg
images.kz/B458SXN.jpg
images.kz/714XAZ05.jpg
images.kz/777NTA01.jpg
images.kz/832EDB02.jpg
images.kz/784EXA01.jpg
images.kz/048LJA10.jpg
images.kz/T985UPM.jpg
images.kz/070OBZ04.jpg
images.kz/E065ETM.jpg
images.kz/168KPA02.jpg
images.kz/191YFA11.jpg
images.kz/211BD01.jpg
images.kz/010FMA04.jpg
images.kz/A335EOO.jpg
images.kz/E614MFM.jpg
images.kz/678LDB13.jpg
images.kz/077TAZ10.jpg
images.kz/446GKB02.jpg
images.kz/564EFA16.jpg
images.kz/064CKA01.jpg
images.kz/956AK10.jpg
images.kz/563CTA13.jpg
images.kz/248HCA11.jpg
images.kz/174EXA04.jpg
images.kz/224SEA02.jpg
images.kz/M463BSN.jpg
images.kz/542CVA11.jpg
i

images.kz/078YFA11.jpg
images.kz/F657KSN.jpg
images.kz/525ANZ08.jpg
images.kz/229DDA03.jpg
images.kz/436EOA14.jpg
images.kz/296PHA16.jpg
images.kz/017DAZ17.jpg
images.kz/491HPA10.jpg
images.kz/259ACA01.jpg
images.kz/494FEA02.jpg
images.kz/295AZ02.jpg
images.kz/186DMA15.jpg
images.kz/362AP10.jpg
images.kz/503GAA09.jpg
images.kz/Z321CDN.jpg
images.kz/729FLA10.jpg
images.kz/421BNA02.jpg
images.kz/186KAZ05.jpg
images.kz/160HVB13.jpg
images.kz/Z520CE.jpg
images.kz/378AAA14.jpg
images.kz/101GGG04.jpg
images.kz/795BWA15.jpg
images.kz/877CLA11.jpg
images.kz/615TMA02.jpg
images.kz/756CLA10.jpg
images.kz/762RC01.jpg
images.kz/360BPA11.jpg
images.kz/663CNA16.jpg
images.kz/318UMZ05.jpg
images.kz/B983NTM.jpg
images.kz/617AD11.jpg
images.kz/M158HNM.jpg
images.kz/T270TTM.jpg
images.kz/204EYA09.jpg
images.kz/334CLA15.jpg
images.kz/A517YFN.jpg
images.kz/P673SLM.jpg
images.kz/045DAZ14.jpg
images.kz/077OMA16.jpg
images.kz/040CMA15.jpg
images.kz/063KXA10.jpg
images.kz/B153XFM.jpg
images.kz/A221NWO.jpg
ima

images.kz/091EXB13.jpg
images.kz/990HNA09.jpg
images.kz/111YDA11.jpg
images.kz/620ZUA02.jpg
images.kz/136FWA04.jpg
images.kz/707BAI03.jpg
images.kz/409KPA16.jpg
images.kz/171VDZ05.jpg
images.kz/644AAA15.jpg
images.kz/417AAZ17.jpg
images.kz/060ALA06.jpg
images.kz/729YA01.jpg
images.kz/476CYA11.jpg
images.kz/010BLA10.jpg
images.kz/992VIP10.jpg
images.kz/179DLA10.jpg
images.kz/355CWA10.jpg
images.kz/A095WCM.jpg
images.kz/Z326RMM.jpg
images.kz/546ETA10.jpg
images.kz/958BM02.jpg
images.kz/602BY02.jpg
images.kz/795EBA07.jpg
images.kz/175XCZ02.jpg
images.kz/F009KTM.jpg
images.kz/303EUA06.jpg
images.kz/252RR02.jpg
images.kz/969ZUA05.jpg
images.kz/141SXA01.jpg
images.kz/154ZVA02.jpg
images.kz/Z797XRM.jpg
images.kz/240UBA16.jpg
images.kz/391KKB05.jpg
images.kz/348YCA13.jpg
images.kz/466KUA10.jpg
images.kz/070KUA04.jpg
images.kz/121FPB02.jpg
images.kz/805LVA01.jpg
images.kz/231DAA11.jpg
images.kz/T309VOM.jpg
images.kz/430DD02.jpg
images.kz/A346FHO.jpg
images.kz/386AOZ01.jpg
images.kz/B060WTN.jpg


images.kz/991HKA07.jpg
images.kz/218FMB05.jpg
images.kz/401CSA15.jpg
images.kz/055DNN01.jpg
images.kz/060AAZ15.jpg
images.kz/911ARB02.jpg
images.kz/699AAZ16.jpg
images.kz/A061ETO.jpg
images.kz/665VCZ15.jpg
images.kz/130CIA09.jpg
images.kz/374EMA08.jpg
images.kz/253TCA02.jpg
images.kz/317PXA01.jpg
images.kz/368CHA11.jpg
images.kz/688XAH10.jpg
images.kz/Z556KTM.jpg
images.kz/D801BY.jpg
images.kz/555AXA04.jpg
images.kz/F150CLN.jpg
images.kz/023BN02.jpg
images.kz/497AKA05.jpg
images.kz/782CVA02.jpg
images.kz/967ALA02.jpg
images.kz/623FOA14.jpg
images.kz/S888APA.jpg
images.kz/134FUA15.jpg
images.kz/258EBA15.jpg
images.kz/Z717YSM.jpg
images.kz/321MAZ14.jpg
images.kz/767FRA15.jpg
images.kz/141NKA12.jpg
images.kz/110LFA10.jpg
images.kz/999GGG10.jpg
images.kz/119YEA11.jpg
images.kz/057YFA11.jpg
images.kz/L610MFM.jpg
images.kz/973MRA05.jpg
images.kz/112PDA05.jpg
images.kz/X911BRO.jpg
images.kz/216COA03.jpg
images.kz/011FLA16.jpg
images.kz/232EHA10.jpg
images.kz/524ABZ10.jpg
images.kz/036CKA10.jp

images.kz/755BFA11.jpg
images.kz/500VVV05.jpg
images.kz/665VRZ03.jpg
images.kz/T689VKM.jpg
images.kz/662YEA11.jpg
images.kz/818OQO02.jpg
images.kz/130UXZ05.jpg
images.kz/095HC01.jpg
images.kz/297FYA10.jpg
images.kz/196KUA10.jpg
images.kz/427DBA11.jpg
images.kz/P599YOM.jpg
images.kz/Z818ZCM.jpg
images.kz/Z573XDM.jpg
images.kz/537HOA10.jpg
images.kz/770OOO09.jpg
images.kz/453LBA10.jpg
images.kz/112DSA11.jpg
images.kz/F270ZRM.jpg
images.kz/016FWB02.jpg
images.kz/916AH01.jpg
images.kz/101ACA02.jpg
images.kz/268FOB02.jpg
images.kz/103AAZ10.jpg
images.kz/A533DUP.jpg
images.kz/579DEA16.jpg
images.kz/X847MM.jpg
images.kz/403FEA10.jpg
images.kz/108XBA05.jpg
images.kz/188AKA11.jpg
images.kz/249CUA06.jpg
images.kz/001AKA04.jpg
images.kz/111TMA01.jpg
images.kz/206AO10.jpg
images.kz/012WMA02.jpg
images.kz/038ACZ16.jpg
images.kz/822KNA10.jpg
images.kz/A254ZKO.jpg
images.kz/101WOA01.jpg
images.kz/542HTA04.jpg
images.kz/735NCA02.jpg
images.kz/784FAA07.jpg
images.kz/136LBA04.jpg
images.kz/362DUA11.jpg
